In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx, to_networkx

In [ ]:
df = pd.read_csv("./data/Nabil Bank Limited ( NABIL ).csv")
df = df.drop(columns=["published_date", "status"])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935 entries, 0 to 2934
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   open             2935 non-null   float64
 1   high             2935 non-null   float64
 2   low              2935 non-null   float64
 3   close            2935 non-null   float64
 4   per_change       2934 non-null   float64
 5   traded_quantity  2935 non-null   float64
 6   traded_amount    2935 non-null   float64
dtypes: float64(7)
memory usage: 160.6 KB


In [14]:
df.describe()

,open,high,low,close,per_change,traded_quantity,traded_amount
count,2935.000000,2935.000000,2935.000000,2935.000000,2934.000000,2935.000000,2.935000e+03
mean,1331.372879,1345.658296,1314.987325,1330.406405,0.024059,28257.086201,3.069377e+07
std,563.054994,570.868204,557.889095,565.095731,2.235349,48984.716252,6.137749e+07
min,429.500000,432.000000,422.000000,424.000000,-26.930000,24.000000,2.164600e+04
25%,848.000000,855.000000,836.000000,845.000000,-0.800000,2401.500000,3.590220e+06
50%,1220.000000,1240.000000,1200.000000,1220.000000,-0.045000,6523.000000,9.077408e+06
75%,1800.000000,1824.000000,1795.000000,1800.000000,0.680000,36814.500000,2.981227e+07
max,2720.000000,2800.000000,2700.000000,2750.000000,20.870000,705430.000000,1.179786e+09


In [ ]:
col_num = len(df.columns)
fig, axes = plt.subplots(col_num, 1, figsize=(6, 4 * col_num), constrained_layout=True)

for i, column in enumerate(df.columns):
    sns.histplot(df[column], kde=True, ax=axes[i])
    axes[i].set_title(f"Histogram of {column} column")
    axes[i].set_xlabel(column)
    axes[i].set_ylabel("Frequency")
plt.show()